In [1]:
from genfft import generate_fft, default_config, fft_two_factor_twiddle
import numpy as np
from copy import copy

In [2]:
cfg = copy(default_config)
cfg["generator_path"] = "../genfft"
cfg["compile_args"] = ["-x", "c++", "-O3", "-include", "genfft/codelet.hh", "-shared"]

In [3]:
fft12 = fft_two_factor_twiddle(cfg, 3, 4)

x = np.arange(12, dtype='complex64')

fft12(x) - np.fft.fft(x)

array([ 0.0000000e+00+0.00000000e+00j,  8.8817842e-16-1.37861639e-06j,
        0.0000000e+00-4.24942073e-07j,  0.0000000e+00+0.00000000e+00j,
        0.0000000e+00-5.39011657e-07j, -8.8817842e-16+4.24942073e-07j,
        0.0000000e+00+0.00000000e+00j, -8.8817842e-16-4.24942073e-07j,
        0.0000000e+00+5.39011657e-07j,  0.0000000e+00+0.00000000e+00j,
        0.0000000e+00+4.24942073e-07j,  8.8817842e-16+1.37861639e-06j])

In [4]:
fft27 = fft_two_factor_twiddle(cfg, 3, 9)
x = np.arange(27, dtype='complex64')
np.abs(fft27(x) - np.fft.fft(x)).max()

8.962143078707292e-06

In [5]:
N = 3
fftN = generate_fft(cfg, "notw_complex", n=N)
x = np.arange(N, dtype='complex64')
y = np.zeros_like(x)
fftN(x, y) 
y

array([ 3. +0.j       , -1.5+0.8660254j, -1.5-0.8660254j], dtype=complex64)

In [4]:
def w(k, n):
    return np.exp(2j * np.pi * k / n)

def make_twiddle(n1, n2):
    I1 = np.arange(n1)
    I2 = np.arange(n2)
    return w(I1[:,None] * I2[None,:], n1*n2).astype('complex64')

def make_fft(config, n, m):
    fft_n = generate_fft(config, "notw_complex", n=n)
    fft_m = generate_fft(config, "twiddle_complex", n=m)
    W = make_twiddle(n, m)[:,1:].copy()
    W1 = make_twiddle(m, n).conj()

    def fft1(x):
        y = np.zeros_like(x).reshape([m, n])
        z = np.zeros_like(x)
        fft_n(x.reshape([n, m]).T, y)
        y *= W1
        fft_m(y.T, z.reshape([m, n]).T)
        return z

    def fft(x):
        y = np.zeros_like(x).reshape([m, n])
        fft_n(x.reshape([n, m]).T, y)
        fft_m(y.T, W)
        return y.flatten()

    return fft

In [5]:
import subprocess

try:
    fft12 = make_fft(cfg, 3, 4)
except subprocess.CalledProcessError as e:
    print(e.stderr)

In [6]:
x = np.arange(12, dtype='complex64')
fft12(x).reshape([4, 3])

notw 3 4 [2, 8] [6, 2]
twiddle 4 3 [2, 6]


array([[66. +0.j       , -6.+22.392303j , -6.+10.392304j ],
       [-6. +6.j       , -6. +3.464101j , -6. +1.6076956j],
       [-6. +0.j       , -6. -1.6076956j, -6. -3.464101j ],
       [-6. -6.j       , -6.-10.392304j , -6.-22.392303j ]],
      dtype=complex64)

In [13]:
np.fft.fft(x).astype('complex64')

array([66. +0.j       , -6.+22.392305j , -6.+10.392304j , -6. +6.j       ,
       -6. +3.4641016j, -6. +1.6076951j, -6. +0.j       , -6. -1.6076951j,
       -6. -3.4641016j, -6. -6.j       , -6.-10.392304j , -6.-22.392305j ],
      dtype=complex64)

In [1]:
!rm lib/*

In [4]:
!pip install sympy

In [5]:
from sympy.ntheory import factorint

In [6]:
factorint(12)

{2: 2, 3: 1}